# Projet de BioInformatique
Rémy Detobel - 000408013                    
rdetobel@ulb.ac.be        

------------------------

## Projet 1        

Le but de ce mini-projet est de créer un algorithme permettant de calculer l'alignement des séquences tout en utilisant une programmation dynamique.          
Ce programme permet d'aligner de manière efficace deux séquences de protéines.  En effet, vu le nombre toujours plus important de données que l'on récupère concernant la génétique, il faut pouvoir classer, stocker et analyser ces données.  Aligner les séquences permet de trouver des similarités et de déterminer s'il existe un ancêtre commun entre les deux séquences.              
Pendant l'évolution, les gênes sont amenés à évoluer.  Lorsqu'une séquence est retranscrite il y a une possibilité que les données soient modifiées.  Certaines modifications n'ont aucune importance, d'autres sont bénéfiques pour l'individu, certaines sont cependant néfastes pour l'organisme et finissent par tuer la cellule (rendant parfois l'individu plus vulnérable voir non viable).            
Aligner les séquences permet donc de détecter ces évolutions ou plutôt les "masquer" pour réussir à découvrir un lien entre deux séquences qui à vue d’œil n'en avaient pas.              

Il faut cependant être attentif aux résultats.  En effet, deux séquences peuvent avoir un très bon alignement mais aucun ancêtre commun.  Il ne s'agit que d'hypothèses qui doivent ensuite être analysées plus en détail pour déterminer s'il y a bien un ancêtre commun.

### Partie 1.1            

Afin de faciliter le développement et de favoriser la maintenance, nous allons créer des ADT (Abstract Data Type) qui nous permettront de manipuler des objets ayant directement des fonctions et des attributs propres à leur contenu.

#### Création d'un objet Séquence        
Une séquence est une liste d'acides aminés.     
Cette séquence continent donc plusieurs attributs: son nom, l'indice de début et de fin (en effet, il ne s'agit que d'une séquence/d'un morceau) et de la séquence elle-même.          
Après initialisation de cette séquence (avec les éléments donnés ci-dessus) il est possible de récupérer sa taille, sa représentation (`string`) ainsi que ses attributs (nom, début, fin).
Il est également possible de récupérer un élément en plein milieu de la séquence.  Pour ce faire, on procède de la même manière qu'avec une liste.

In [1]:
class Sequence:

    # Création de l'objet
    def __init__(self, sequence = [], seqNom = "", debut = -1, fin = -1):
        self.sequence = list(sequence)
        self.debut = debut
        self.fin = fin
        self.nom = seqNom

    # Récupérer sa longueur
    def __len__(self):
        return len(self.sequence)

    # Récupérer un élément à un indice specifique
    def __getitem__(self, item):
        return self.sequence[item]

    # Permet de définir un élément à un indice specifique
    # Une séquence ne sera jamais modifiée (dans ce programme)
    # def __setitem__(self, key, item):
    #     self.sequence[key] = item

    # Transformer la séquence en string
    def __str__(self):
        res = ""
        for lettre in self.sequence:
            res += lettre;
        return res

    # Permet d'avoir l'indice de début de la séquence
    def getDebut(self):
        return self.debut

    # Permet d'avoir l'indice de fin de la séquence
    def getFin(self):
        return self.fin

    # Permet d'avoir le nom de la séquence
    def getNom(self):
        return self.nom

    # Permet de comparer la séquence actuelle avec une autre
    def isEgal(self, sequence):
        egal = True
        i = 0

        if(len(self.sequence) != len(sequence)):
            egal = False

        while(egal and i < len(self.sequence)):
            egal = (sequence[i] == self.sequence[i])
            i+=1
            
        return egal

#### Création d'un objet Score
L'objet Score va contenir une `matrice de substitution`.  Elle permet de connaitre les scores de similarités entre toutes les paires d'acides aminés.      
Il existe deux grands types de matrices de subsitution: PAM et BLOSUM.         
- PAM              
  La matrice PAM a été construite en utilisant de vraies séquences similaires.  Il existe plusieurs matrices se différenciant par le nombre de substitution qu'elles acceptent.                   
- BLOSUM             
  La matrice BLOSUM a été construite grâce à des blocs d'acides aminés bien conservés dans les protéines.  Là également, il existe plusieurs matrices.  Le projet 2 parle d'ailleurs plus en détails des matrices de substitution.                           

Le nombre indiqué à la suite du nom de la matrice, par exemple le `62` dans la matrice: `BLOSUM 62`, représente la précision d'identité qui a été utilisée pour créer cette matrice.         

Concrètement, notre objet `Score` va donc stocker cette matrice.  La seule chose que l'on ait besoin c'est la valeur qu'a cette matrice pour deux acides aminés donnés.  On implémente donc une méthode `get` permettant de récupérer cette valeur.

In [2]:
class Score:

    # Création de l'objet
    def __init__(self, nom_fichier):
        # Initialisation des variables
        self.listLettre = {}
        self.matrice = []

        # Ouverture du fichier
        fichier = open(nom_fichier, 'r')

        nbrLigne = 0
        for ligne in fichier:
            if(len(ligne) > 0 and ligne[0] != '#'):
                # Séparer la ligne et supprimer les espaces
                ligne = ligne.strip().split()

                # Vérification de la définition de la liste des lettres
                if(len(self.listLettre) == 0):
                    # Si le premier caractère est une lettre
                    if(ligne[0].isalpha()):
                        self.listLettre = dict(enumerate(ligne))
                        self.listLettre = {v: k for k, v in self.listLettre.items()}

                    # Sinon, le fichier n'est pas correct
                    else:
                        raise SyntaxError("Fichier invalide (debug ligne: %s )" % ligne)

                else:
                    self.matrice.append(ligne)
                    ++nbrLigne

                    
    # Permet de récupérer le score en fonction de deux lettres
    def get(self, lettre1, lettre2):
        number1 = self.listLettre[lettre1]
        number2 = self.listLettre[lettre2]
        return self.matrice[max(number1, number2)][min(number1, number2)]



##### Charger les matrices de substitution
Pour charger les matrices de substitution il suffit de passer en paramètre le chemin vers un fichier `.iij`.  L'objet `Score` s'occupera de parser le fichier et de créer la matrice.                
Si il manque la liste des lettres composant la matrice, une exception `SyntaxError` sera levée.

### Partie 1.2 et 1.3
Ces deux parties vont être présentées en même temps vu que la partie 1.3 modifie le code de la partie 1.2.      
Jusqu'à présent, nous avons seulement créer des ADT qui ne "font" rien tout seul.  En effet, il faut les initialiser, leur donner des informations et les faire travailler ensemble.

#### Création d'un objet Align
Afin de structurer le code, on a créé un objet `Align`.  Cet objet contiendra plusieurs variables de travail mais également le coeur de l'application.  En effet, c'est dans cet objet que l'on va calculer le meilleur alignement en fonction des paramètres donnés à la création de l'objet.                 
L'objet `Align` peut donc faire trois types d'alignement:
- L'alignement Global
- L'alignement Semi-Global
- L'alignement Local              

Il permet de calculer l'alignement avec une pénalité affine.  Cependant, pour faire une pénalité linéaire il suffit de mettre deux fois la même valeur (pour l'ouverture d'un trou et l'extention de celui-ci).           
Pour faciliter son utilisation, l'objet `Align` dispose de valeur par défaut.  Les seules paramètres obligatoires sont les deux chaines ainsi que le Score.        
Le dernier paramètre permet également d'activer un mode "debug" permettant d'afficher beaucoup plus d'informations.
           
          
C'est lors de la création de l'objet que tous les calculs sont faits.  On va d'abord initialiser 3 matrices permettant de stocker (plus tard) les valeurs calculées en ajoutant les pénalités et les trous.  Ces matrices sont initialisées de différentes manières en fonction des paramètres.  Pour les matrices semiGlobal et local on met des 0 sur la ligne et la colonne 0.  On va également initialiser une matrice `deplacement` qui stockera tous les déplacements possibles à partir d'une case.       
      
Une fois les matrices initialisées, on doit les remplir.  Pour ce faire on utilise les valeurs au-dessus, à gauche et en diagonale de la case courante.           
On va donc remplir les 3 matrices avec les formules suivantes:             
$$
\begin{eqnarray}
V(i, j) &=& max((S[i-1][j]-I), V[i-1][j]-E)\\
W(i, j) &=& max((S[i][j-1]-I), W[i][j-1]-E)\\
S(i, j) &=& max((S[i-1][j-1] + score), (V[i][j]), (W[i][J])) \\
\end{eqnarray}
$$              
i est la ligne, j la colonne, E la pénalité pour étendre un trou et I la pénalité pour initialiser un trou.         
Les matrices V et W nous permettent donc de retenir les pénalités où un trou a déjà été initialisé tandis que S est la matrice que l'on va utiliser pour faire notre parcours.           
Afin d'optimiser le programme, on calcule directement les directions possibles.  Pour se faire on regarde laquelle des valeurs possibles pour S a été choisie.  On enregistre les différents déplacements possibles dans la matrice `deplacement` (initialisée précédemment).  On aura donc dans une case i,j les déplacements possibles (soit `H`orizontal, soit `V`ertical, soit `D`iagonale).          
On profite également de cette construction pour enregistrer la valeur la plus grande de la matrice ou de la dernière ligne (respectivement pour l'alignement local ou semi-global).          

Enfin, on appelle une fonction qui va parcourir la matrice `deplacement` afin de trouver les chemins les plus optimisés.  Comme expliqué précédemment en case (i, j), on aura la lettre H et/ou V et/ou D qui serviront à savoir si l'on peut se déplacer vers le haut, la gauche ou en diagonale.       
Pour l'alignement global on part de la case dans le coin inférieur droit et on s'arrête lorsque l'on arrive en 0, 0.  Pour l'alignement semi-global c'est la même chose mais l'on part de la plus grande valeur sur la dernière ligne.             
Pour l'alignement local il est possible qu'il y ait plusieurs points de départ.  En effet, on part de la valeur la plus élevée de la matrice.  Cette valeur peut apparaitre plusieurs fois dans la matrice.      
Dans tous les cas, on enregitre tous les chemins possibles (respectants les conditions citées précédemment).

In [3]:
class Align:    
    
    # Création de l'objet
    def __init__(self, chaineA, chaineB, score, openPenal = -12, extendPenal = -2, \
                 local = False, semiGlobal = False, debug = False):


        # Initialisation des variables
        self.chaineA = chaineA
        self.chaineB = chaineB
        self.score = score
        self.openPenal = openPenal     # Coût pour ouvrir un trou
        self.extendPenal = extendPenal # Coût pour prolonger un trou
        self.local = local
        self.semiGlobal = semiGlobal
        self.debug = debug
        
        # Initialisation des matrices de calcul
        self.S = []
        self.V = []
        self.W = []
        # Creation d'une matrice ou la liste en ij permettra d'indiquer dans quelle
        # direction on peut se deplacer (Horizontale, Verticale ou Diagonale)
        self.deplacement = []
        
        # Valeur maximum du tableau
        self.maxValue = 0
        # Coordonnees où l'on peut trouver cette valeur
        self.maxI = [] 
        self.maxJ = []

        self.initMatrice()       # Initialisation de la matrice
        self.remplisageMatrice() # Remplissage de la matrice
        self.res = []   # Résultat final
        self.minIJ = [] # Coordonnées les plus petites atteintes

        
        # Parcours de la matrice pour trouver le plus cours chemin
        if(self.local or self.semiGlobal): # Si c'est une recherche locale ou semi-globale
            for allMax in range(len(self.maxI)): # Parcourt toutes les valeurs maximum
                self.res += (self.findMinPath(self.maxI[allMax], self.maxJ[allMax], [], []))

        else: # Sinon on fait une simple recherche
            self.res = self.findMinPath(len(chaineA), len(chaineB))



    # Permet d'initialiser la matrice
    def initMatrice(self):
        
        for i in range(len(self.chaineA)+1):
            # Initialisation des lignes
            self.S.append([])
            self.V.append([])
            self.W.append([])
            self.deplacement.append([])
            
            for j in range(len(self.chaineB)+1):
                if(i == 0): # Si l'on est sur la première ligne
                    if(self.local or self.semiGlobal):
                        valeur = 0
                        self.deplacement[i].append([])
                    else:
                        # La valeur va être égale à la colonne fois l'ouverture d'une pénalité si
                        # l'on est en colonne 0 ou 1, sinon ce sera égale à la ligne précedente 
                        # plus le prolongement de la pénalité
                        valeur = j*self.openPenal if j <= 1 else self.S[i][j-1] + self.extendPenal
                        # Les déplacements se font horizontalement sauf en 0,0
                        self.deplacement[i].append(["H"] if j != 0 else ["H", "V"])
                        
                    # Initialisation des 3 matrices
                    self.S[i].append(valeur)
                    self.V[i].append(valeur)
                    self.W[i].append(valeur)

                elif(j == 0): # Si l'on est sur la première colonne
                    if(self.local or self.semiGlobal):
                        valeur = 0 # par défaut on met 0 partout
                        self.deplacement[i].append([])
                    else:
                        # Pareil que pour la ligne 0 mais ici c'est la colonne
                        valeur = i*self.openPenal if i <= 1 else self.S[i-1][j] + self.extendPenal
                        self.deplacement[i].append(["V"])
                    
                    # Initialisation des 3 matrices
                    self.S[i].append(valeur)
                    self.V[i].append(valeur)
                    self.W[i].append(valeur)

                else: # Sinon on remplit juste la liste des déplacements
                    self.deplacement[i].append([])

    # Permet d'afficher une matrice
    def printMatrice(self, matrice):
        for i in range(len(matrice)):
            # Ligne avec les indices
            if(i == 0):
                print("     ", end="")
                for j in range(len(matrice[i])):
                    print('{:4}|'.format(j), end="")
                print()
            
            # Affichage des lignes
            for j in range(len(matrice[i])):
                if(j == 0):
                    # Affiche l'indice
                    print('{:4}|'.format(i), end="")

                if(isinstance(matrice[i][j], list)):
                    print('{:4}|'.format("".join(matrice[i][j])), end="")
                else:
                    print('{:4}|'.format(str(matrice[i][j])), end="")
            print() # Retour à la ligne

    
    # Permet de trouver le(s) bon(s) chemin(s) dans la matrice
    def findMinPath(self, i, j, current=[], sol=[]):
        # Si on a fini le parcours
        if(i == 0 and j == 0 or (self.semiGlobal and (i == 0 or j == 0))):
            sol.append(list(reversed(current)))
            self.minIJ.append([i, j])

        else:
            continuer = False # Est-il possible de continuer
            
            # Si on peut encore monter et que ce déplacement est possible
            if(i > 0 and "V" in self.deplacement[i-1][j]):
                continuer = True
                current.append("V") # Construction d'une solution
                sol = self.findMinPath(i-1, j, current[:], sol)
                current = current[:-1] # Déconstruction
            
            # Test à gauche cette fois
            if(j > 0 and "H" in self.deplacement[i][j-1]):
                continuer = True
                current.append("H")
                sol = self.findMinPath(i, j-1, current[:], sol)
                current = current[:-1]

            # Et enfin en diagonale
            if(i > 0 and j > 0 and "D" in self.deplacement[i-1][j-1]):
                continuer = True
                current.append("D")
                sol = self.findMinPath(i-1, j-1, current[:], sol)
                current = current[:-1]
            
            # Si il n'est plus possible de continuer et qu'on est en local
            if(not continuer and self.local):
                # On enregistre donc la solution
                sol.append(list(reversed(current)))
                self.minIJ.append([i, j])

        return sol


    # Permet de remplir la matrice
    def remplisageMatrice(self):
        # On parcourt toute la matrice
        for i in range(1, len(self.chaineA)+1):
            for j in range(1, len(self.chaineB)+1):
                
                # V(i, j) = max((S[i-1][j]-I), V[i-1][j]-E)
                choixV1 = self.S[i-1][j]+self.openPenal
                choixV2 = self.V[i-1][j]+self.extendPenal

                self.V[i].append(max(choixV1, choixV2))

                # W(i, j) = max((S[i][j-1]-I), W[i][j-1]-E)
                choixW1 = self.S[i][j-1]+self.openPenal
                choixW2 = self.W[i][j-1]+self.extendPenal

                self.W[i].append(max(choixW1, choixW2))

                # S(i, j) = max((S[i-1][j-1] + score), (V[i][j]), (W[i][J]))
                choixS1 = self.S[i-1][j-1] + int(self.score.get(self.chaineA[i-1], self.chaineB[j-1]))
                choixS2 = self.V[i][j]
                choixS3 = self.W[i][j]

                # Si on cherche les séquences locales
                if(self.local):
                    maximum = max(choixS1, choixS2, choixS3, 0)
                else:
                    maximum = max(choixS1, choixS2, choixS3)

                # Enregistrement de la valeur maximum si l'on est pas en
                # semiGlobal ou que l'on est sur la dernière ligne
                if(not self.semiGlobal or i == len(self.chaineA)):
                    # Si la valeur est la même que le max actuel
                    if(self.maxValue == maximum):
                        self.maxI.append(i)
                        self.maxJ.append(j)

                    # Si le max est plus grand que la valeur actuelle
                    elif(maximum > self.maxValue):
                        self.maxI = [i]
                        self.maxJ = [j]
                        self.maxValue = maximum
                        

                # On ajout le maximum dans la matrice
                self.S[i].append(maximum)

                
                if(self.S[i][j] == choixS1):
                    self.deplacement[i-1][j-1].append("D") # Diagonale

                if(self.S[i][j] == choixS2):
                    self.deplacement[i-1][j].append("V") # Horizontal

                if(self.S[i][j] == choixS3):
                    self.deplacement[i][j-1].append("H") # Vertical
        
        if(self.debug): # Si on veut débug, on affiche les matrices intermédiaires
            self.printMatrice(self.S)
            print()
            self.printMatrice(self.deplacement)
    
    
    # Permet d'afficher toutes les solutions
    def printResult(self):
        # Parcours de toutes les solutions
        for indexDeplacement in range(len(self.res)):
            # Initialisation des variables de travail
            resSeq1 = self.chaineA.getNom()
            separation = " "
            resSeq2 = self.chaineB.getNom()

            # Aligne les 3 champs
            tailleEspace = max(len(resSeq1), len(resSeq2))
            resSeq1 += (tailleEspace - len(resSeq1))*separation + ": "
            resSeq2 += (tailleEspace - len(resSeq2))*separation + ": "
            separation = " "*tailleEspace + "  "

            
            # colonne et ligne de début à 0 sauf si on est en local
            col = 0 if(not (self.local or self.semiGlobal)) else self.minIJ[indexDeplacement][0]
            ligne =  0 if(not (self.local or self.semiGlobal)) else self.minIJ[indexDeplacement][1]

            if(self.debug):
                print("Solution: " + str(self.res[indexDeplacement]))

            # On parcourt le résultat
            for dep in self.res[indexDeplacement]:

                if dep == "D":
                    resSeq1 += self.chaineA[col]
                    resSeq2 += self.chaineB[ligne]
                    # Affiche un double point si les lettres sont les mêmes
                    if(self.chaineA[col] == self.chaineB[ligne]):
                        separation += ":"
                    # Un simple point si le score est positif
                    elif(int(self.score.get(self.chaineA[col], self.chaineB[ligne])) >=  0):
                        separation += "."
                    # Un espace si rien
                    else:
                        separation += " "
                        
                    # On déplace le curseur
                    ligne += 1
                    col += 1

                elif dep == "H":
                    resSeq1 += "-"
                    resSeq2 += self.chaineB[ligne]
                    separation += " "
                    ligne += 1

                elif dep == "V":
                    resSeq1 += self.chaineA[col]
                    resSeq2 += "-"
                    separation += " "
                    col += 1

            # Affichage des solutions
            print(resSeq1)
            print(separation)
            print(resSeq2)
            print("")
            

    def getResult(self):
        return self.res

###### Affichage des réponses            
Pour afficher les réponses, on parcourt tous les résultats.  Concernant l'alignement local, on ne peut pas partir assurément des coordonnées 0, 0.  Lors du parcours on aura donc enregistré les coordonnées les plus petites atteintes.  C'est donc à partir de ces coordonnées que l'on va commencer notre parcours du haut à gauche vers le bas à droite.

#### Charger les séquences
Les séquences sont égalements stockées dans un fichier.  Un fichier contient plusieurs séquences.  Il n'est donc pas judicieux de mettre cette fonction dans l'objet `Sequence` (vu que cette fonction crée plusieurs objets `Sequence`).  Les fichiers utilisés pour importer des séquences sont des fichiers `.fasta`.          
En cas de problème dans les informations d'une séquence, une exception `SyntaxError` est levée.

In [4]:
def chargerFichierSequence(nom_fichier):
    res = [] # Liste où seront stockées les séquences

    # Ouverture du fichier
    fichier = open(nom_fichier, 'r')

    nomSequence = "" # Nom de la séquence
    debut = -1       # Index de début de séquence
    fin = -1         # Index de fin de séquence
    sequence = ""    # La séquence en elle-même

    # On lit chaque ligne du fichier
    for ligne in fichier:
        ligne = ligne.strip() # Supprime les espaces inutiles
        if(ligne[0] == '>'): # début d'une séquence

            if(sequence != ""): # Si la séquence n'est pas vide
                # On enregistre
                res.append(Sequence(sequence, nomSequence, debut, fin))
                # On remet tout à zero
                nomSequence = ""
                debut = -1
                fin = -1
                sequence = ""

            # On extrait les informations
            infos = ligne.split('|')
            if(len(infos) == 3):
                nomSequence = infos[1]
                debutFin = infos[2].split("-")

                if(len(debutFin) == 2):
                    debut = debutFin[0]
                    fin = debutFin[1]
                    
            else: # Si on n'a pas toutes les informations on informe qu'il y a une erreur
                raise SyntaxError("Fichier invalide (debug ligne: %s )" % ligne)

        else: # Suite d'une séquence
            sequence += ligne.strip()

    # On n'oublie pas la dernière séquence (qui n'a aucune autre séquence après elle)
    if(sequence != ""): 
        res.append(Sequence(sequence, nomSequence, debut, fin))

    return res

#### Quelques tests de ces objets/fonctions
Maintenant que tous les objets et les fonctions ont été définis, on peut réaliser un alignement
                  
                       
###### Premier Test
On va donc effectuer un premier test.  Il s'agit ici d'un des exemples présenté au TP.  On charge donc le fichier `seqTest.fasta` (ce fichier contient plusieurs séquences utilisées comme exemple au cours ou en TP) dans lequel on va prendre les deux premières séquences.   On affiche ces séquences ainsi que leur taille respective.              
On charge la matrice BLOSUM60 grâce au fichier `blosum60.iij`.      
Enfin, on réalise un alignement global avec une pénalité de -4 (pour la création d'un nouveau trou) et -1 (pour le prolongement d'un trou existant), tout en affichant les résultats.           
**Résultats:** les résultats de cet alignement sont similaires à ceux présentés lors de la scéance TP.

In [5]:
##### TEST 1 #####
listSeq = chargerFichierSequence("seqTest.fasta")

seq0_0 = listSeq[0]
seq0_1 = listSeq[1]
print(seq0_0.getNom() + ": " + str(seq0_0) + " (taille: " + str(len(seq0_0)) + ")")
print(seq0_1.getNom() + ": " + str(seq0_1) + " (taille: " + str(len(seq0_1)) + ")")
print("")

blosum60 = Score("./BLOSUM/blosum60.iij")
align = Align(seq0_0, seq0_1, blosum60, -4, -1)
align.printResult()

Test0: THISLINE (taille: 8)
Test1: ISALIGNED (taille: 9)

Test0: THIS-LI-NE-
         :: :: :: 
Test1: --ISALIGNED



###### Second test
On charge toujours les mêmes séquences (dans le cas présent, il est inutile de refaire le chargement du fichier.  Cependant cette opération a été répétée ici pour permettre de déplacer facilement les modules).      
Cette fois-ci nous allons faire un alignement semi-global avec une pénalité linéaire de `-8`.  Il s'agit d'un exemple fait dans les slides.           
**Résultats:** Les résultats sont identiques à ceux présents dans le cours.

In [6]:
##### TEST 2 #####
listSeq1 = chargerFichierSequence("seqTest.fasta")

seq1_0 = listSeq1[0]
seq1_1 = listSeq1[1]
print(seq1_0.getNom() + ": " + str(seq1_0) + " (taille: " + str(len(seq1_0)) + ")")
print(seq1_1.getNom() + ": " + str(seq1_1) + " (taille: " + str(len(seq1_1)) + ")")
print("")

blosum62 = Score("./BLOSUM/blosum62.iij")
align1 = Align(seq1_0, seq1_1, blosum62, -8, -8, False, True)
align1.printResult()            

Test0: THISLINE (taille: 8)
Test1: ISALIGNED (taille: 9)

Test0: IS-LI-NE
       :: :: ::
Test1: ISALIGNE



###### Troisième test
Cette fois ci, nous allons utilier le fichier `SH3-sequence.fasta` mis à notre disposition pour ce projet.  Comme dans les tests précédents, nous prennons les deux premières séquences.            
On charge également la matrice BLOSUM62.            
Pour ce second test, nous allons faire un alignement semi-global avec les mêmes pénalités que lors du premier test (à savoir -4 pour commencer un trou et -1 pour le prolonger)

In [7]:
##### TEST 3 #####
listSeq2 = chargerFichierSequence("SH3-sequence.fasta")

seq2_0 = listSeq2[0]
seq2_1 = listSeq2[1]
print(seq2_0.getNom() + ": " + str(seq2_0) + " (taille: " + str(len(seq2_0)) + ")")
print(seq2_1.getNom() + ": " + str(seq2_1) + " (taille: " + str(len(seq2_1)) + ")")
print("")

blosum62 = Score("./BLOSUM/blosum62.iij")
align1 = Align(seq2_0, seq2_1, blosum62, -4, -1, False, True)
align1.printResult()

P12931: GGVTTFVALYDYESRTETDLSFKKGERLQIVNNTEGDWWLAHSLSTGQTGYIPSNYVAPSDS (taille: 62)
P62993: MEAIAKYDFKATADDELSFKRGDILKVLNEECDQNWYKAELNGKDGFIPKNYIEMKPH (taille: 58)

P12931: F--VALYDYESRTETDLSFKKGERLQIVNNTEGD--WWLAHSLSTGQTGYIPSNYVAPSDS
        .  .: ::... .. .::::.:. :...:. : :  :. :. :. :. :.::.::.   . 
P62993: MEAIAKYDFKATADDELSFKRGDILKVLNE-ECDQNWYKAE-LN-GKDGFIPKNYI---EM

P12931: F--VALYDYESRTETDLSFKKGERLQIVNNTEGD--WWLAHSLSTGQTGYIPSNYVAPSDS
        .  .: ::... .. .::::.:. :...:. : :  :. : .:. :. :.::.::.   . 
P62993: MEAIAKYDFKATADDELSFKRGDILKVLNE-ECDQNWYKA-ELN-GKDGFIPKNYI---EM

P12931: F--VALYDYESRTETDLSFKKGERLQIVNNTEGD--WWLAHSLSTGQTGYIPSNYVA--PSDS
        .  .: ::... .. .::::.:. :...:. : :  :. :. :. :. :.::.::.   :   
P62993: MEAIAKYDFKATADDELSFKRGDILKVLNE-ECDQNWYKAE-LN-GKDGFIPKNYIEMKP---

P12931: F--VALYDYESRTETDLSFKKGERLQIVNNTEGD--WWLAHSLSTGQTGYIPSNYVA--PSDS
        .  .: ::... .. .::::.:. :...:. : :  :. : .:. :. :.::.::.   :   
P62993: MEAIAKYDFKATADDELSFKRGDILKVLNE-ECDQNWYKA-ELN

###### Quatrième test
Dans ce quatrième test, nous allons chercher l'alignement local de deux séquences présentes dans SH3.  Pour ce test nous allons utiliser les valeurs par défaut proposées par le système [LALIGN](http://www.ch.embnet.org/software/LALIGN_form.html), c'est à dire 12 et 2 comme pénalité d'ouverture et de prolongement de trou (respectivement).  Voici le résultat calculé par LALIGN:
![title](res_local.png)                    
**Solutions:** La solution calculée correspond bien à celle de LALIGN.

In [8]:
##### TEST 4 #####
listSeq3 = chargerFichierSequence("SH3-sequence.fasta")

seq2_0 = listSeq3[0]
seq2_1 = listSeq3[1]
print(seq2_0.getNom() + ": " + str(seq2_0) + " (taille: " + str(len(seq2_0)) + ")")
print(seq2_1.getNom() + ": " + str(seq2_1) + " (taille: " + str(len(seq2_1)) + ")")
print("")

blosum50 = Score("./BLOSUM/blosum50.iij")
align2 = Align(seq2_0, seq2_1, blosum50, -12, -2, True)
align2.printResult()
print("")

P12931: GGVTTFVALYDYESRTETDLSFKKGERLQIVNNTEGDWWLAHSLSTGQTGYIPSNYVAPSDS (taille: 62)
P62993: MEAIAKYDFKATADDELSFKRGDILKVLNEECDQNWYKAELNGKDGFIPKNYIEMKPH (taille: 58)

P12931: VALYDYESRTETDLSFKKGERLQIVNNTEGDWWLAHSLSTGQTGYIPSNYV
        .: ::... .. .::::.:. :...:.   . :    :. :. :.::.::.
P62993: IAKYDFKATADDELSFKRGDILKVLNEECDQNWYKAELN-GKDGFIPKNYI




##### Test avec les fichiers `maguk`


In [9]:
##### TEST Maguk 1 #####
listSeqMaguk = chargerFichierSequence("maguk-sequences.fasta")

seqMaguk0_0 = listSeqMaguk[0]
seqMaguk0_1 = listSeqMaguk[1]
print(seqMaguk0_0.getNom() + ": " + str(seqMaguk0_0) + " (taille: " + str(len(seqMaguk0_0)) + ")")
print(seqMaguk0_1.getNom() + ": " + str(seqMaguk0_1) + " (taille: " + str(len(seqMaguk0_1)) + ")")
print("")

blosum50 = Score("./BLOSUM/blosum90.iij")
align2 = Align(seqMaguk0_0, seqMaguk0_1, blosum50, -12, -5, True)
align2.printResult()
print("")

Q12959: MPVRKQDTQRALHLLEEYRSKLSQTEDRQLRSSIERVINIFQSNLFQALIDIQEFYEVTLLDNPKCIDRSKPSEPIQPVNTWEISSLPSSTVTSETLPSSLSPSVEKYRYQDEDTPPQEHISPQITNEVIGPELVHVSEKNLSEIENVHGFVSHSHISPIKPTEAVLPSPPTVPVIPVLPVPAENTVILPTIPQANPPPVLVNTDSLETPTYVNGTDADYEYEEITLERGNSGLGFSIAGGTDNPHIGDDSSIFITKIITGGAAAQDGRLRVNDCILRVNEVDVRDVTHSKAVEALKEAGSIVRLYVKRRKPVSEKIMEIKLIKGPKGLGFSIAGGVGNQHIPGDNSIYVTKIIEGGAAHKDGKLQIGDKLLAVNNVCLEEVTHEEAVTALKNTSDFVYLKVAKPTSMYMNDGYAPPDITNSSSQPVDNHVSPSSFLGQTPASPARYSPVSKAVLGDDEITREPRKVVLHRGSTGLGFNIVGGEDGEGIFISFILAGGPADLSGELRKGDRIISVNSVDLRAASHEQAAAALKNAGQAVTIVAQYRPEEYSRFEAKIHDLREQMMNSSISSGSGSLRTSQKRSLYVRALFDYDKTKDSGLPSQGLNFKFGDILHVINASDDEWWQARQVTPDGESDEVGVIPSKRRVEKKERARLKTVKFNSKTRDKGEIPDDMGSKGLKHVTSNASDSESSYRGQEEYVLSYEPVNQQEVNYTRPVIILGPMKDRINDDLISEFPDKFGSCVPHTTRPKRDYEVDGRDYHFVTSREQMEKDIQEHKFIEAGQYNNHLYGTSVQSVREVAEKGKHCILDVSGNAIKRLQIAQLYPISIFIKPKSMENIMEMNKRLTEEQARKTFERAMKLEQEFTEHFTAIVQGDTLEDIYNQVKQIIEEQSGSYIWVPAKEKL (taille: 904)
Q92796: MHKHQHCCKCPECYEVTRLAALRRLEPPGYGDWQVPDPYGPGGGNGASAGYGGYSSQTLPSQAGA

Via le site [UNIPROT](http://www.uniprot.org/) on peut remarquer que toutes ces séquences ont un lien avec `PDZ`.  On remarque également que toutes ces séquences proviennent de l'homme.